<a href="https://colab.research.google.com/github/saravanakumarjsk/Simple-torch-scripts-/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
device = 'cuda'



In [6]:
import torchvision
from torchvision import datasets
import torchvision.transforms as t

transforms = t.Compose([t.RandomHorizontalFlip(),
                       t.RandomRotation(10),
                       t.ToTensor(),
                       t.Normalize((0.5, 0.5, 0.5),
                                  (0.5, 0.5, 0.5))])

traindata = datasets.CIFAR10('data', train = True, download = True, transform = transforms)
testdata = datasets.CIFAR10('data', train = False, download = True, transform = transforms)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
train_len = len(traindata)
print(train_len)

50000


In [0]:
index = list(range(train_len))
np.random.shuffle(index)
split = int(np.floor(train_len * 0.2))
train_index, val_index = index[split:], index[:split]

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler as s

train_sampler = s(train_index)
valid_sampler = s(val_index)

In [0]:
train_loader = torch.utils.data.DataLoader(traindata, 
                                           batch_size = 16,
                                           sampler = train_sampler,
                                          num_workers = 4)

valid_loader = torch.utils.data.DataLoader(traindata, 
                                          batch_size = 16,
                                          sampler = valid_sampler,
                                          num_workers = 4)


test_loader = torch.utils.data.DataLoader(testdata, 
                                           batch_size = 16,
                                          num_workers = 4)

In [0]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

In [0]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# # helper function to un-normalize and display an image
# def imshow(img):
#     img = img / 2 + 0.5  # unnormalize
#     plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image


# # plot the images in the batch, along with the corresponding labels
# fig = plt.figure(figsize=(25, 4))
# # display 20 images
# for idx in np.arange(20):
#     ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
#     imshow(images[idx])
#     ax.set_title(classes[labels[idx]])



In [0]:
import torch.nn as nn
import torch.nn.functional as f

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2,2)
        #64*4*4
        self.fc1 = nn.Linear(1024, 500)
        self.fc2 = nn.Linear(500, 10)
        
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = self.pool(f.relu(self.conv3(x)))
        
        x = x.view(-1, 1024)
        x = self.dropout(x)
        
        x = f.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        

In [14]:
model = Net()
model.to(device)

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [0]:
val_loss_min = np.Inf

for epoch in range(1, 40):
    
    val_loss = 0.0
    train_loss = 0.0
    model.train()
    
    for idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        model.eval()
    for idx, (images, labels) in enumerate(valid_loader):
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        
        val_loss += loss.item()
        
    train_loss = train_loss/len(train_loader.sampler)
    val_loss = val_loss/len(valid_loader.sampler)
 

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, val_loss))
    
    # save model if validation loss has decreased
    if val_loss <= val_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        val_loss_min,
        val_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        val_loss_min = val_loss

Epoch: 1 	Training Loss: 0.126553 	Validation Loss: 0.106421
Validation loss decreased (inf --> 0.106421).  Saving model ...
Epoch: 2 	Training Loss: 0.097543 	Validation Loss: 0.089855
Validation loss decreased (0.106421 --> 0.089855).  Saving model ...
Epoch: 3 	Training Loss: 0.086630 	Validation Loss: 0.084010
Validation loss decreased (0.089855 --> 0.084010).  Saving model ...
Epoch: 4 	Training Loss: 0.079189 	Validation Loss: 0.078108
Validation loss decreased (0.084010 --> 0.078108).  Saving model ...
Epoch: 5 	Training Loss: 0.073029 	Validation Loss: 0.073278
Validation loss decreased (0.078108 --> 0.073278).  Saving model ...
Epoch: 6 	Training Loss: 0.067658 	Validation Loss: 0.069557
Validation loss decreased (0.073278 --> 0.069557).  Saving model ...
Epoch: 7 	Training Loss: 0.063272 	Validation Loss: 0.062688
Validation loss decreased (0.069557 --> 0.062688).  Saving model ...
Epoch: 8 	Training Loss: 0.059665 	Validation Loss: 0.062275
Validation loss decreased (0.06268

In [0]:
model.load_state_dict(torch.load('model_augmented.pt'))

In [0]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for batch_idx, (data, target) in enumerate(test_loader):
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))